### Number of protests within given distance
Number of protests that happened in the past week within a given ditance. This is using the weekly dataset, where all locations are recorded every week, with an indicator for whether a protest happened here or not

In [7]:
import pandas as pd
import numpy as np

from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [8]:
data = pd.read_csv('data/blm_weekly_protests.csv')
distances = pd.read_csv('data/dist_matrix_geoid.csv')
distances = distances.set_index('GEOID10')

In [9]:
protest_list = data['Protest?'].to_list()

In [10]:
split_list = [protest_list[x:x+3601] for x in range(0, len(protest_list), 3601)]

## Protests under a certain distance
Matrix multiplication using the distance matrix and the column reocrding protest occurrence

In [11]:
under_500 = distances.copy()
under_500[0<under_500<=500] = 1
under_500[under_500>500] = 0

protests_under_500 = []

for i in range(0, 159):
    protests_under_500.append(np.dot(split_list[i], under_500).tolist())
    
zeros = [0 for i in range(0, 3601)]
protests_under_500.insert(0, zeros)
protests_under_500 = protests_under_500[0:159]

flat_500_list = [i for a in protests_under_500 for i in a]

In [6]:
under_1000 = distances.copy()
under_1000[0<under_1000<=1000] = 1
under_1000[under_1000>1000] = 0

protests_under_1000 = []

for i in range(0, 159):
    protests_under_1000.append(np.dot(split_list[i], under_1000).tolist())
    
zeros = [0 for i in range(0, 3601)]
protests_under_1000.insert(0, zeros)
protests_under_1000 = protests_under_1000[0:159]

flat_1000_list = [i for a in protests_under_1000 for i in a]

In [7]:
under_2000 = distances.copy()
under_2000[0<under_2000<=2000] = 1
under_2000[under_2000>2000] = 0

protests_under_2000 = []

for i in range(0, 159):
    protests_under_2000.append(np.dot(split_list[i], under_2000).tolist())
    
zeros = [0 for i in range(0, 3601)]
protests_under_2000.insert(0, zeros)
protests_under_2000 = protests_under_2000[0:159]

flat_2000_list = [i for a in protests_under_2000 for i in a]

## Adding number of protests for each week and making list that can then be appended as column

In [8]:
sum_list = [sum(split_list[i]) for i in range(0, len(split_list))]

In [9]:
sum_for_df = [val for val in sum_list for _ in range(0, 3601)]

In [10]:
zeros = [0 for i in range(0, 3601)]

In [11]:
final_sum = zeros + sum_for_df # need a week lag
final_sum = final_sum[0:572559]

## Same state

In [12]:
data['State'] = data['NAME10'].str.split(',').str[1]
data['State'] = data['State'].str.strip()

In [13]:
state_list = data['State'].to_list()

In [14]:
split_state_list = [state_list[x:x+3601] for x in range(0, len(state_list), 3601)]

In [15]:
list_lists = []

for a, b in zip(range(0, 159), range(0, 159)):
    new_list = []
    for c, d in zip(split_list[a], split_state_list[b]):
        if c == 0:
            pass
        elif c == 1:
            new_list.append(d)
    list_lists.append(new_list)

In [16]:
empty = []
list_lists.insert(0, empty)
list_lists = list_lists[0:159]

In [17]:
dicts = [dict(Counter(list_lists[i])) for i in range(0, 159)]

In [18]:
protests_per_state = []

for state in state_list:
    try:
        protests_per_state.append(dicts[23][state])
    except:
        protests_per_state.append(0)

In [19]:
protests_per_state = []

for i, j in zip(range(0, 159), range(0, 159)):
    for a in split_state_list[i]:
        try:
            protests_per_state.append(dicts[j][a])
        except:
            protests_per_state.append(0)

## Currently not accounting for states that are hyphenated (should sum value of any where two characters overlap - do this in dict?)

# Adding all columns created to the df

In [20]:
data['Protests in past week'] = final_sum
data['Protests in same state past week'] = protests_per_state
data['Protests under 500km past week'] = flat_500_list
data['Protests under 1000km past week'] = flat_1000_list
data['Protests under 2000km past week'] = flat_2000_list

In [21]:
data.to_csv('data/final_weekly_data.csv')